In [ ]:
!pip3 install -q langchain openai pandas
import langchain
import openai
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import get_openai_callback

In [ ]:
key = ''

In [ ]:
global costs
global iters
costs = 0
iters = 1

def parse_cost(cost):
    return float(str(cost).split("\n")[-1].split(" ")[-1].replace("$",""))

In [ ]:
import os

base_dir = "/kaggle/input/melee-questions-firstpass/"

dfs = [pd.read_csv(base_dir+x) for x in os.listdir(base_dir)]

df = pd.concat(dfs)
df = df.dropna()
df = df[df['qa'].str.contains('Q:')].reset_index()[['character','qa']]
df = df.sample(frac=1)

In [ ]:
characters = ['Captain Falcon','Falco','Fox','Ice Climbers','Jigglypuff','Marth','Peach','Sheik']

In [ ]:
df['character'].value_counts()

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named 'df' and a list of characters named 'characters'
#characters = ['Captain Falcon', 'Falco', 'Fox', 'Ice Climbers', 'Jigglypuff', 'Marth', 'Peach', 'Sheik']

def remove_apostrophe(text):
    return text.replace("'", "").lower()

# Ensure not dealing with non-character related things. General non-character-specific items I can add later maybe
df = df[df['qa'].apply(lambda text: any(remove_apostrophe(char) in remove_apostrophe(text) for char in characters))]


In [ ]:
model_gpt35 = "gpt-3.5-turbo-16k"
chatgpt = ChatOpenAI(temperature = 0.7, model = model_gpt35, openai_api_key = key)

In [ ]:
qa_review_prompt_template="""
You are a Questions and Answers (Q and A) reviewer.
Your task is to evaluate the content of the Q and A about character #CHARACTER# from Super Smash Bros Melee created by ChatGPT based on the following criteria:

1. There is no mention of outside entities that are not strictly featured in Super Smash Brothers Melee.
2. Any characters mentioned in the Q and A that are not #CHARACTER# are only included in relation to the matchup between the character and #CHARACTER#.
3. Q must mention #CHARACTER#
4. There is no mention of guides in Q or A

State which critiera DO NOT apply.

##Q and A##
#CHARACTER#: captain_falcon
Q: How does Helios think he would stand up against those players now?
A: Helios stated that he has no idea how he would stand up against those players now.'

##Q and A Evaluation##:
2,4


##Q and A##
#CHARACTER#: fox
Q: How can Fox use his shine for edge-guarding?
A: Fox's shine can be used for edge-guarding by positioning himself at the edge of the stage and activating the shine when the opponent nears the ledge. This can disrupt the opponent's recovery and potentially lead to a shine spike or edge-guard opportunity. By shining the opponent from the ledge, Fox can maintain control and apply pressure to prevent them from safely returning to the stage. Following up with a grab or shine spike can lead to a stock advantage for Fox.

##Q and A Evaluation##:
1,2,3,4


##Q and A##:
#CHARACTER#: jigglypuff
Q: Can Pikachu combo opponents with Uairs?
A: While Pikachu cannot combo opponents with Uairs, he can still lay on the damage, so players should not get too comfortable in the air. ',

##Q and A Evaluation##:
1,4


##Q and A##:
#CHARACTER#: sheik
Q: What should I keep in mind when approaching and prioritizing against Donkey Kong as Sheik?
A: When approaching and prioritizing against Donkey Kong as Sheik, it is important to refer to the guide for specific tips and strategies on this matchup. The guide provides valuable information on how to effectively approach and prioritize against DK.

##Q and A Evaluation##:
1,2,3
[Reason: Criterion 4 fail, guide mentioned]

##Q and A##:
#CHARACTER#: sheik
Q: What stages does the guide recommend striking as Falco against Fox?
A: The guide recommends striking Pokemon Stadium and Fountain of Dreams as stages for Falco against Fox. Pokemon Stadium is not ideal because Fox's shine allows him to drop as low as he wants for his kill, and his recovery lets him get away with it. Fountain of Dreams' platforms get in the way of Falco's short hop to laser.

##Q and A Evaluation##:
1,2,3,4

Your turn..
##Q and A##:
#CHARACTER#: {character}
{qa}

##Q and A Evaluation##:

"""

In [ ]:
outside_entity_prompt = """
You are a Questions and Answers (Q and A) reviewer.
Your task is to evaluate the content of the Q and A about character #CHARACTER# from Super Smash Bros Melee created by ChatGPT and decide where there is mention of outside entities that do not appear in Super Smash Brothers Melee for the Nintendo Gamecube.

If this is true, reply with fail
If this is untrue, reply with pass

##Q and A Evaluation##:

##Q and A##
#CHARACTER#: {character}
{qa}

##Q and A Evaluation##:

"""

In [ ]:
includes_character_prompt = """
How many names mentioned are characters in Super Smash Bros Melee?

##Q and A##
{qa}
"""

In [ ]:
no_guides_prompt= """
You are a helpful assistant that removes mentions of a guide or source of information from a Q and A.

##Q and A##
Q: What stages does the guide recommend striking as Falco against Fox? 
A: The guide recommends striking Pokemon Stadium and Fountain of Dreams as stages for Falco against Fox. Pokemon Stadium is not ideal because Fox's shine allows him to drop as low as he wants for his kill, and his recovery lets him get away with it. Fountain of Dreams' platforms get in the way of Falco's short hop to laser.

##Fixed Q and A##
Q: What stages should Falco strike against Fox? 
A: You should consider striking Pokemon Stadium and Fountain of Dreams as stages for Falco against Fox. Pokemon Stadium is not ideal because Fox's shine allows him to drop as low as he wants for his kill, and his recovery lets him get away with it. Fountain of Dreams' platforms get in the way of Falco's short hop to laser.

##Q and A##
{qa}

##Fixed Q and A##

"""

In [ ]:
includes_character_template = ChatPromptTemplate.from_messages([
    ("system", includes_character_prompt),
])

outside_entity_template = ChatPromptTemplate.from_messages([
    ("system", outside_entity_prompt),
])

no_guides_template = ChatPromptTemplate.from_messages([
    ("system", no_guides_prompt),
])

In [ ]:

def vet_outside_entity(character, qa):
    global costs
    global iters
    print(f'Progress: {iters}/{len(df)}')
    iters += 1
    with get_openai_callback() as cost:
        outside_entity_test = chatgpt(outside_entity_template.format_messages(character = character, qa = qa)).content
    costs += parse_cost(cost)
    print(costs)
    return outside_entity_test

In [ ]:
no_guide = df[~df['qa'].str.contains('guide')]
guide = df[df['qa'].str.contains('guide')]

In [ ]:
i = 1
new_qa = []
for index, row in guide.iterrows():
    qa_to_change = row['qa']
    print(f'Progress: {i}/{len(guide)}')
    i += 1
    with get_openai_callback() as cost:
        new_qa.append(chatgpt(no_guides_template.format_messages(qa = qa_to_change)).content)
        costs += parse_cost(cost)
        print(costs)


In [ ]:
guide['qa'] = new_qa

In [ ]:
df = pd.concat([no_guide, guide])

In [ ]:
df['vetted'] = df.apply(lambda x: vet_outside_entity(x['character'],x['qa']), axis = 1)

In [ ]:
df.sample(frac=1).to_csv('polished_final.csv')